In [2]:
# Predicting House Prices in Australia
# Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.metrics import mean_absolute_error

In [3]:
# Call created by inserting data file to code
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_67f476715e2f4a92b07bba55beba4ee6 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='cghZDWwzp8Q9Nk76wKTqIftpsT7g0RrEz14jzBR7bO58',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_67f476715e2f4a92b07bba55beba4ee6.get_object(Bucket='machinelearningforabsolutebeginne-donotdelete-pr-9hkcr7lauqnidf',Key='Melbourne_housing_FULL.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [4]:
# Learn more about the dataframe
df.shape
df.iloc[100]
df.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [5]:
# Delete unneeded columns
del df['Address']
del df['Method']
del df['SellerG']
del df['Date']
del df['Postcode']
del df['Lattitude']
del df['Longtitude']
del df['Regionname']
del df['Propertycount']

In [6]:
# View leftover columns
df.columns

Index(['Suburb', 'Rooms', 'Type', 'Price', 'Distance', 'Bedroom2', 'Bathroom',
       'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea'],
      dtype='object')

In [7]:
# Remove rows with any missing values, with no threshold on count of missing values, 
# with no subset of columns, and inplace (update, rather than replace) true
df.dropna(axis = 0, how ='any', thresh = None, subset = None, inplace = True)

In [8]:
# Count the number of rows in dataframe after deleting missing values
df.count

<bound method DataFrame.count of            Suburb  Rooms Type      Price  Distance  Bedroom2  Bathroom  Car  \
2      Abbotsford      2    h  1035000.0       2.5       2.0       1.0  0.0   
4      Abbotsford      3    h  1465000.0       2.5       3.0       2.0  0.0   
6      Abbotsford      4    h  1600000.0       2.5       3.0       1.0  2.0   
11     Abbotsford      3    h  1876000.0       2.5       4.0       2.0  0.0   
14     Abbotsford      2    h  1636000.0       2.5       2.0       1.0  2.0   
...           ...    ...  ...        ...       ...       ...       ...  ...   
34847     Wollert      3    h   500000.0      25.5       3.0       2.0  2.0   
34849     Wollert      3    h   570000.0      25.5       3.0       2.0  2.0   
34853  Yarraville      2    h   888000.0       6.3       2.0       2.0  1.0   
34854  Yarraville      2    t   705000.0       6.3       2.0       1.0  2.0   
34856  Yarraville      2    h  1020000.0       6.3       2.0       1.0  0.0   

       Landsize  B

In [ ]:
# For non-numeric columns, determine how many unique values there are before one-hot encoding
# One-hot encoding is replacing categorical variables with dummy binary variables before building
# machine-learning model
df['Suburb'].nunique()
df['Type'].nunique()
df['CouncilArea'].nunique()

In [9]:
# See the datatype for each dataframe column
df.dtypes
# Suburb, CouncilArea and Type columns return object for string. One-hot encode these columns
# Other columns are int or float
df = pd.get_dummies(df,columns = ['Suburb','CouncilArea','Type'])

In [10]:
df.dtypes
# Assign independent variables to X, and dependent variables to y with Price
X = df.drop('Price',axis=1)
y = df['Price']

In [11]:
# Shuffle and split the dataset to train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,shuffle=True)

In [30]:
# Select model algorithm and configure hyperparameters
model = ensemble.GradientBoostingRegressor(
n_estimators = 150,
learning_rate = 0.1,
max_depth = 30,
min_samples_split = 4,
min_samples_leaf = 6,
max_features = 0.6,
loss = 'huber'
)

In [31]:
# Fit model to training set
model.fit(X_train,y_train)

GradientBoostingRegressor(loss='huber', max_depth=30, max_features=0.6,
                          min_samples_leaf=6, min_samples_split=4,
                          n_estimators=150)

In [34]:
# Determine training set MAE. On average, how much has price being miscalculated in the training set?
mae_train = mean_absolute_error(y_train, model.predict(X_train))
print("Training Set Mean Absolute Error: %.2f"% mae_train)

Training Set Mean Absolute Error: 28636.48


In [35]:
# Determine test set MAE. On average, how much has price being miscalculated in the test set?
mae_test = mean_absolute_error(y_test, model.predict(X_test))
print("Test Set Mean Absolute Error: %.2f"% mae_test)

Test Set Mean Absolute Error: 171669.51


In [36]:
# Low training set MAE and high test set MAE means that our models are overfitted and 
# we must now optimize the model

In [20]:
# MODEL OPTIMIZATION
# 1. Adjust maximum depth hyperparameter in model.Change from 30 to 5
model2 = ensemble.GradientBoostingRegressor(
n_estimators = 150,
learning_rate = 0.1,
max_depth = 5,
min_samples_split = 4,
min_samples_leaf = 6,
max_features = 0.6,
loss = 'huber'
)
model2.fit(X_train,y_train)
mae2_train = mean_absolute_error(y_train, model2.predict(X_train))
mae2_test = mean_absolute_error(y_test, model2.predict(X_test))

In [21]:
print(mae2_train)
print(mae2_test)

135562.4660685072
163748.8041647579


In [22]:
# 2. Adjust n_estimators hyperparameter in model.Change from 150 to 250
model3 = ensemble.GradientBoostingRegressor(
n_estimators = 250,
learning_rate = 0.1,
max_depth = 5,
min_samples_split = 4,
min_samples_leaf = 6,
max_features = 0.6,
loss = 'huber'
)
model3.fit(X_train,y_train)
mae3_train = mean_absolute_error(y_train, model3.predict(X_train))
mae3_test = mean_absolute_error(y_test, model3.predict(X_test))

In [23]:
print(mae3_train)
print(mae3_test)

123861.00665119664
158123.1239655676


In [27]:
# 3. Try grid search
model4 = ensemble.GradientBoostingRegressor()
hyperparameters = {
    'n_estimators': [200,300],
    'learning_rate': [0.01,0.02],
    'max_depth': [4,6],
    'min_samples_split': [3,4],
    'min_samples_leaf': [5,6],
    'max_features': [0.8,0.9],
    'loss': ['ls','lad','huber']
}
# Defne grid search & run with 4 CPUs in parallel
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(model4,hyperparameters,n_jobs=4)

In [28]:
# Run grid search on training data
grid.fit(X_train,y_train)
# Return optimal hyperparameters
grid.best_params_
# Check model accuracy with determined optimal hyperparameters
mae4_train = mean_absolute_error(y_train,grid.predict(X_train))
mae4_test = mean_absolute_error(y_test,grid.predict(X_test))

In [29]:
print(mae4_train)
print(mae4_test)

141147.2308050096
173506.3952639447


In [ ]:
# Even after doing a grid search for a Gradient Boosting Regressor's optimal
# hyperparameters, the mae's are still pretty high